# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import json
import azureml.core
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.model import Model
from azureml.core import Workspace, Experiment, Model, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
# from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools



## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml-exp'

experiment=Experiment(ws, experiment_name)

In [7]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute


aml_compute_cluster = "heart-compute" #can not be more than 16 chars

# Verify that cluster does not exist already
try:
    # compute_target = ComputeTarget(ws, aml_compute_cluster)
# if aml_compute_cluster in ws.compute_targets:
    # compute_target = ws.compute_targets[aml_compute_cluster]
    compute_target = ComputeTarget(workspace=ws, name=aml_compute_cluster)
    print('Found existing cluster, use it.')
except: 
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', min_nodes=1,max_nodes=4)
    compute_target = ComputeTarget.create(ws, aml_compute_cluster, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
print(compute_target.get_status().serialize())

Creating
Succeeded........................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-07T11:22:21.687000+00:00', 'errors': None, 'creationTime': '2021-02-07T11:20:12.214362+00:00', 'modifiedTime': '2021-02-07T11:20:28.832260+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [8]:

key = "HeartFailure-Dataset"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [20]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             **automl_settings
)

In [21]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on heart-compute with default configuration
Running on remote compute: heart-compute
Parent Run ID: AutoML_64549a3f-4f93-4138-801f-5a1c3da07036

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing value

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [23]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [24]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_64549a3f-4f93-4138-801f-5a1c3da07036',
 'target': 'heart-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T12:16:08.352557Z',
 'endTimeUtc': '2021-02-07T12:34:59.307735Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'heart-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"auto-ml-exp","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-137807","workspace_name":"quick-starts-ws-137807","region":"southcentralus","compute_target":"heart-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":nul

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [26]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [28]:
print('Best Run Id: ', best_run.id)
# print('\n Accuracy: ', best_run_metrics['Accuracy'])
print('\n Metrics: ', best_run_metrics)
print('\n Best Model: ', fitted_model)

Best Run Id:  AutoML_64549a3f-4f93-4138-801f-5a1c3da07036_39

 Metrics:  {'average_precision_score_macro': 0.8799327921525731, 'AUC_weighted': 0.8961027036889107, 'AUC_micro': 0.906675652144339, 'balanced_accuracy': 0.8189736534564122, 'f1_score_micro': 0.8561616161616162, 'precision_score_macro': 0.8517341287496704, 'average_precision_score_weighted': 0.9041730906372276, 'matthews_correlation': 0.6685559766468868, 'average_precision_score_micro': 0.9061423533312474, 'AUC_macro': 0.8961027036889105, 'recall_score_micro': 0.8561616161616162, 'norm_macro_recall': 0.6379473069128242, 'precision_score_weighted': 0.860262491542239, 'recall_score_macro': 0.8189736534564122, 'recall_score_weighted': 0.8561616161616162, 'weighted_accuracy': 0.8857230122409777, 'log_loss': 0.4154179981108936, 'f1_score_macro': 0.8262168825116042, 'precision_score_micro': 0.8561616161616162, 'accuracy': 0.8561616161616162, 'f1_score_weighted': 0.8506077867009405, 'accuracy_table': 'aml://artifactId/ExperimentRun

In [29]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('robustscaler',
                                                            RobustScaler(copy=True,
                                                                         quantile_range=[10,
                                                                                         90],
                                                                         with_centering=True,
                                                                         with_scaling=False)),
                                                           ('randomforestclassifier',
                                                            RandomForestClassifier(bootstrap=True,
                                                                                   ccp_alpha=0.0,
         

In [30]:
#TODO: Save the best model
automl_best_model = best_run.register_model(model_name = "best-automl-model", model_path = './outputs/model.pkl', properties={'Accuracy': best_run_metrics['accuracy']})
print(automl_best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-137807', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-137807'), name=best-automl-model, id=best-automl-model:2, version=2, tags={}, properties={'Accuracy': '0.8561616161616162'})


In [31]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for props_name in model.properties:
        props = model.properties[props_name]
        print ('\t',props_name, ':', props)
    print('\n')

best-automl-model version: 2
	 Accuracy : 0.8561616161616162


best_hyperdrive_model version: 1


best-automl-model version: 1
	 Accuracy : 0.8594949494949495




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
os.makedirs('./atoml-model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./atoml-model','best-automl-model.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./atoml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [ ]:

model=best_run.register_model(
            model_name = 'best-automl-model', 
            model_path = './outputs/model.pkl'
)

In [ ]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'environment_file.yml')

In [ ]:

# Create an inference config 
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=best_run.get_environment())

# Deploy the model as a web service
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True, enable_app_insights=True)
webservice = Model.deploy(ws, "myservice", [automl_best_model], inference_config, aci_config)
webservice.wait_for_deployment(show_output = True)


In [ ]:
webservice

In [ ]:
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#test data

test_df = df.sample(5) 
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

In [ ]:
output = webservice.run(test_sample)
print(output)

In [ ]:
# Send a request to the web service you deployed to test it.

import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, test_sample, headers=headers)

In [ ]:
print(response.text)

In [ ]:
print(label_df)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(webservice.get_logs())

In [ ]:
webservice.delete()